# Alice, Bob & Eve: Symmetric Cryptography

## Alice

Alice generates her symmetric key, which is fairly easy, as it it just a bunch of random bits. 256 of them to be precise.

In [22]:
var rng = System.Security.Cryptography.RandomNumberGenerator.Create();
var secretKey = new byte[32]; // 32 bytes = 256 bits
rng.GetBytes(secretKey);
Console.WriteLine($"Generated Secret Key: {Convert.ToBase64String(secretKey)}");

Generated Secret Key: wbMTlVo/yxVi2uCJf2m3+S8s3tu4jl4eqkuuBqjaKro=


Next, Alice encrypts her message using AES GCM (Galois Counter Mode). She uses GCM, because she is smart and therefore uses authenticated encryption. If you aren't smart, I'll explain what AE is in separate video someday.

In [23]:
var message = "I love you!";
var messageBytes = System.Text.Encoding.UTF8.GetBytes(message);

var aliceAes = new System.Security.Cryptography.AesGcm(secretKey, 16);
var nonce = new byte[12];
rng.GetBytes(nonce);
var cipherText = new byte[messageBytes.Length];
var tag = new byte[16];
aliceAes.Encrypt(nonce, messageBytes, cipherText, tag);

Console.WriteLine($"Nonce:       {Convert.ToBase64String(nonce)}");
Console.WriteLine($"Tag:         {Convert.ToBase64String(tag)}");
Console.WriteLine($"Cipher Text: {Convert.ToBase64String(cipherText)}");

Nonce:       CxgYLdpd3YEorjc6
Tag:         AI0JELBtBhI1sv+cC6SxWw==
Cipher Text: yOz2cX5jYmM//h0=


Then Alice joins the values together into a single data structure:

In [24]:
var messageData = nonce
    .Concat(tag)
    .Concat(cipherText)
    .ToArray();
Console.WriteLine($"Message Data: {Convert.ToBase64String(messageData)}");

Message Data: CxgYLdpd3YEorjc6AI0JELBtBhI1sv+cC6SxW8js9nF+Y2JjP/4d


## Bob

Bob has two thigs:

1. The secret key, obtained by secure means.
2. The message data structure, containing nonce, tag and ciphertext.

In [25]:
Console.WriteLine($"Message Data: {Convert.ToBase64String(messageData)}");
Console.WriteLine($"Secret Key:   {Convert.ToBase64String(secretKey)}");

Message Data: CxgYLdpd3YEorjc6AI0JELBtBhI1sv+cC6SxW8js9nF+Y2JjP/4d
Secret Key:   wbMTlVo/yxVi2uCJf2m3+S8s3tu4jl4eqkuuBqjaKro=


First, bob will parse the message data structure. He knows that nonce is first 12 bytes and tag is next 16 bytes, so the rest must be the ciphertext:

In [26]:
var parsedNonce = messageData.Take(12).ToArray();
var parsedTag = messageData.Skip(12).Take(16).ToArray();
var parsedCipherText = messageData.Skip(28).ToArray();

Console.WriteLine($"Nonce:       {Convert.ToBase64String(parsedNonce)}");
Console.WriteLine($"Tag:         {Convert.ToBase64String(parsedTag)}");
Console.WriteLine($"Cipher Text: {Convert.ToBase64String(parsedCipherText)}");

Nonce:       CxgYLdpd3YEorjc6
Tag:         AI0JELBtBhI1sv+cC6SxWw==
Cipher Text: yOz2cX5jYmM//h0=


Now he will create instance of `AesGcm` class and will feed it the parameters he has:

In [27]:
var bobAes = new System.Security.Cryptography.AesGcm(secretKey, 16);
var decryptedBytes = new byte[parsedCipherText.Length];
bobAes.Decrypt(parsedNonce, parsedCipherText, parsedTag, decryptedBytes);
var decryptedMessage = System.Text.Encoding.UTF8.GetString(decryptedBytes);
Console.WriteLine($"Decrypted Message: {decryptedMessage}");

Decrypted Message: I love you!


It works! But it has several shortcomings:

1. We need secure channel to transport the secret key so Eve cannot eavesdrop it. Which may be quite hard and if we have such a channel, why don't use it for the entire message?
2. We need to generate new key for each and every message, because if we use the same symmetric key twice, it means that Eve can break the encryption.

We can tackle the second problem with using key derivation. We can solve the first problem using asymmetric alhorithms, like RSA, DHM or ML-KEM. We'll look into this later.